# An In-depth Evaluation of Approaches to Text Classification (IDEATC)

## II. Classic Supervised Baselines

_This notebook is used to establish baselines using "classic" supervised learning approaches to text classification, including Naive Bayes, Logistic Regression and Support Vector Machines._

### Libraries

In [1]:
# standard library
import os
from pathlib import Path

# data wrangling
import numpy as np
import datasets

# local packages
import src
from src.experiments import supervised
from src.frameworks.sklearn import models, optimisation

# other settings
LOAD_PATH_DATASET = Path(os.pardir, 'data', 'processed')
SAVE_PATH_RESULTS = Path(os.pardir, 'data', 'results')
src.utils.show_datasets(LOAD_PATH_DATASET)

,dataset,num_rows_train,num_rows_valid,num_rows_test,num_classes
0,yelp_review_full_processed,650000,0,2500,5
1,yelp_polarity_processed,560000,0,1000,2
2,dbpedia_14_processed,560000,0,7000,14
3,ag_news_processed,120000,0,7600,4
4,web_of_science_processed,37589,0,9396,134
5,imdb_processed,25000,0,1000,2
6,dynabench_dynasent_processed,13065,720,720,3
7,20_newsgroups_processed,11314,0,7532,20
8,setfit_sst5_processed,8544,1101,2210,5
9,rotten_tomatoes_processed,8530,1066,1066,2


## II. Dummy Classifier

In [2]:
for path in LOAD_PATH_DATASET.glob('*processed*'):
    dataset = datasets.load_from_disk(path)
    sample_sizes = src.experiments.utils.get_sample_sizes(dataset['train'])
    supervised.run_experiment(
        dataset_dict=dataset,
        feature='text_clean',
        get_model=models.get_dummy_model,
        search_params={},
        optimisation=optimisation,
        sample_sizes=sample_sizes,
        progress_bar=True,
        experiment_id='dummy_classifier',
        save_path=SAVE_PATH_RESULTS.joinpath(path.name),
    )
print('Done!')

Training size: 256 Running grid search...:   0%|          | 0/9 [00:00<?, ?it/s]/opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
Training size: 512 Running grid search...:  11%|█         | 1/9 [00:00<00:03,  2.18it/s]    /opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
Training size: 1,024 Running grid search...:  22%|██▏       | 2/9 [00:00<00:02,  2.38it/s]  /opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
Training size: 2,048 Running grid search...:  33%|███▎      | 3/9 [00:01<00:02,  2.40it/s]    /opt/anaconda3/

Done!


## III. Naive Bayes

In [2]:
params_vectoriser = {
    'vectoriser__min_df': [2, 5],
    'vectoriser__max_features': [10_000, 30_000, 50_000],
    'vectoriser__binary': [True, False],
}

params_clf = {
    'clf__alpha': [.1, .5, 1, 2, 5]  # Laplace smoothing
}
params = params_vectoriser | params_clf

In [4]:
for path in LOAD_PATH_DATASET.glob('*processed*'):
    dataset = datasets.load_from_disk(path)
    sample_sizes = src.experiments.utils.get_sample_sizes(dataset['train'])
    supervised.run_experiment(
        dataset_dict=dataset,
        feature='text_clean',
        get_model=models.get_naive_model,
        search_params=params,
        optimisation=optimisation,
        sample_sizes=sample_sizes,
        progress_bar=True,
        experiment_id='complement_naive_bayes',
        save_path=SAVE_PATH_RESULTS.joinpath(path.name),
    )
print('Done!')

Training size: 8,192 Running grid search...:  71%|███████▏  | 5/7 [00:26<00:12,  6.23s/it]    /opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Training size: 11,314 Running grid search...:  86%|████████▌ | 6/7 [00:41<00:09,  9.13s/it]   /opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Training size: 65,536 Running grid search...:  80%|████████  | 8/10 [00:21<00:08,  4.28s/it]    /opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This 

Done!


## IV. Logistic Regression / SVM

In [3]:
params_clf = {
    'clf__loss': ['hinge', 'log_loss'],
    'clf__alpha': np.logspace(-6, 1, 8),  # i.e., 1e-6 to 10
}
params = params_vectoriser | params_clf

In [ ]:
for path in LOAD_PATH_DATASET.glob('*processed*'):
    dataset = datasets.load_from_disk(path)
    sample_sizes = src.experiments.utils.get_sample_sizes(dataset['train'])
    supervised.run_experiment(
        dataset_dict=dataset,
        feature='text_clean',
        get_model=models.get_linear_model,
        search_params=params,
        optimisation=optimisation,
        sample_sizes=sample_sizes,
        progress_bar=True,
        experiment_id='sgd_classifier',
        save_path=SAVE_PATH_RESULTS.joinpath(path.name),
    )
print('Done!')

Training size: 8,192 Running grid search...:  71%|███████▏  | 5/7 [01:10<00:35, 17.75s/it]    /opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/sklearn/pipeline.py:359: UserWarning: Persisting input arguments took 0.62s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(
/opt/anaconda3/envs/ocom5300m/lib/python3.9/site-packages/sklearn/pipeline.py:359: UserWarning: Persisting input arguments took 0.67s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB

## V. Sanity Check

In [22]:
for path in LOAD_PATH_DATASET.glob('*_processed'):
    df_metrics = src.experiments.utils.read_metrics(SAVE_PATH_RESULTS, path.name)
    fig = src.plotting.plot_performance_overall(df_metrics=df_metrics, add_se=True)
    fig.update_layout(title=path.name)
    fig.show()